<a href="https://colab.research.google.com/github/mnansary/pyF2O/blob/master/colab_gen_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab specific task
*   Check TF version
*   mount google drive
*   TPU check
*   Change Working Directory to cloned dir 

NOTE: **git clone the repo before running this notebook**

In [0]:
!pip3 install tensorflow==1.14.0

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:

# tpu check
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', TPU_ADDRESS)

  with tf.Session(TPU_ADDRESS) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)


In [0]:
cd /content/gdrive/My\ Drive/PROJECTS/F2O/pyF2O/

# GCS specific task 
* **auth user**
* **save** and **upload** credentials to **tpu**
* set project information


In [0]:
# auth user for cloud SDK
from google.colab import auth
auth.authenticate_user()

In [0]:
# Save credentials
import json
SERVICE_KEY_PATH='/content/adc.json' # @param
# Upload credentials to TPU.
with tf.Session(TPU_ADDRESS) as sess:    
    with open(SERVICE_KEY_PATH, 'r') as f:
        auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)
# set service_account
JSON_DATA=json.load(open(SERVICE_KEY_PATH))
SERVICE_ACCOUNT=str(JSON_DATA['client_id']).split('.')[0]
print('Service Account:',SERVICE_ACCOUNT)

#### SET PROJECT INFORMATION 

In [0]:
PROJECT_ID    ='f2oitpu'               # @param 
BUCKET        ='f2odata'               # @param 

# LIST FILES
TFRECORDS_DIR= 'gs://{}/{}/'.format(BUCKET,'tfrecord')
MODEL_DIR    = 'gs://{}/{}/'.format(BUCKET,'model_dir')
# change TFRECORDS_DIR specific to structre
#
!gcloud config set project {PROJECT_ID}
!gsutil ls {TFRECORDS_DIR}

# pix2pix
* set **FLAGS** 
* define **model function**
* define **input function**
* define **config**
* define **estimator**

#### FLAGS AND PARAMS

In [0]:
class FLAGS:
    IMAGE_DIM       = 256       # @param
    NB_CHANNELS     = 3         # @param
    BATCH_SIZE      = 128       # @param
    SHUFFLE_BUFFER  = 1000      # @param    
    NB_TOTAL_DATA   = 16128     # @param
    NB_EVAL_DATA    = 3200      # @param
    ITERATIONS      = 500       # @param
    LEARNING_RATE   = 0.0002    # @param

MODEL_EXPORT_NAME = 'f20-pix2pix' # @param
STEPS           = FLAGS.NB_TOTAL_DATA // FLAGS.BATCH_SIZE 
print('Num of Steps:{}'.format(STEPS))


#### Model Function

In [0]:
from F2O.pix2pix import loss_fn

def model_fn(features, labels, mode, params):
    if mode == tf.estimator.ModeKeys.PREDICT:
        raise RuntimeError("mode {} is not supported yet".format(mode))

    loss = loss_fn(features, labels)

    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(
                                    FLAGS.LEARNING_RATE,
                                    tf.train.get_global_step(),
                                    decay_steps=100000,
                                    decay_rate=0.96
                        )
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

        return tf.contrib.tpu.TPUEstimatorSpec(
                                                mode=mode,
                                                loss=loss,
                                                train_op=optimizer.minimize(
                                                    loss, 
                                                    tf.train.get_global_step()
                                                    )
                                              )

#### Data Input Functions
 

In [0]:
from google.cloud import storage
from functools import partial

client = storage.Client(PROJECT_ID)
# get bucket from the project
bucket=client.get_bucket(BUCKET)
print(bucket)

def data_input_fn(FLAGS,params): 
    
    def _parser(example):
        feature ={  'image'  : tf.io.FixedLenFeature([],tf.string) ,
                    'target' : tf.io.FixedLenFeature([],tf.string)
        }    
        parsed_example=tf.io.parse_single_example(example,feature)
        image_raw=parsed_example['image']
        image=tf.image.decode_png(image_raw,channels=FLAGS.NB_CHANNELS)
        image=tf.cast(image,tf.float32)/255.0
        image=tf.reshape(image,(FLAGS.IMAGE_DIM,FLAGS.IMAGE_DIM,FLAGS.NB_CHANNELS))
        
        target_raw=parsed_example['target']
        target=tf.image.decode_png(target_raw,channels=FLAGS.NB_CHANNELS)
        target=tf.cast(target,tf.float32)/255.0
        target=tf.reshape(target,(FLAGS.IMAGE_DIM,FLAGS.IMAGE_DIM,FLAGS.NB_CHANNELS))
        
        return image,target

    #dataset = tf.data.TFRecordDataset([os.path.join('gs://{}/'.format(BUCKET), f.name) for f in bucket.list_blobs(prefix='tfrecord/{}'.format(mode))])
    dataset = tf.data.TFRecordDataset([os.path.join('gs://{}/'.format(BUCKET), f.name) for f in bucket.list_blobs()])
    dataset = dataset.cache()
    dataset = dataset.shuffle(FLAGS.SHUFFLE_BUFFER,reshuffle_each_iteration=True)
    dataset = dataset.map(_parser)
    dataset = dataset.repeat()
    dataset = dataset.batch(FLAGS.BATCH_SIZE,drop_remainder=True)
    dataset = dataset.prefetch(-1) # autotune 
    return dataset

train_input_fn = partial(data_input_fn,FLAGS=FLAGS)

def serving_input_fn():
    inputs = {"serving_input": tf.placeholder(tf.float32,[None,FLAGS.IMAGE_DIM,FLAGS.IMAGE_DIM,FLAGS.NB_CHANNELS])}  
    features = inputs['serving_input']  
    return tf.estimator.export.TensorServingInputReceiver(features, inputs)  
    

#### TPU config and estimator

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)
 
TPU = tf.contrib.cluster_resolver.TPUClusterResolver()
 
config = tf.contrib.tpu.RunConfig(
                                   cluster=TPU,
                                   model_dir=MODEL_DIR,
                                   tpu_config=tf.contrib.tpu.TPUConfig(STEPS)
                                   )
estimator = tf.contrib.tpu.TPUEstimator(
                                        model_fn=model_fn,
                                        model_dir=MODEL_DIR,
                                        train_batch_size=FLAGS.BATCH_SIZE,
                                        config=config,
                                        use_tpu=True,
                                        export_to_tpu=False
                                        )  
# we want an exported model for CPU/GPU inference 
estimator.train(train_input_fn, steps=STEPS*FLAGS.ITERATIONS)
estimator.export_savedmodel(os.path.join(MODEL_DIR, MODEL_EXPORT_NAME), serving_input_fn)